# **Deep Learning With Python  -  CHAPTER 12**

- This code implements a **Generative Adversarial Network (GAN)** to generate realistic face images using the **CelebA dataset**.

- It is structured into several classes for better modularity:
  
  `CelebAGANDataLoader` handles dataset downloading and preprocessing, `Discriminator` and `Generator` define the two neural networks, `GAN` encapsulates the training logic, and `GANMonitor` saves generated images during training.

- The model is trained using **binary cross-entropy loss**, and the generator learns to create increasingly realistic images by competing with the discriminator. The training loop runs for **100 epochs**, generating images at the end of each epoch.

In [2]:
import os
import pathlib
import random
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [3]:
class CelebAGANDataLoader:
    def __init__(self, dataset_path="celeba_gan", image_size=(64, 64), batch_size=32):
        self.dataset_path = dataset_path
        self.image_size = image_size
        self.batch_size = batch_size

    def download_and_extract_data(self):
        os.makedirs(self.dataset_path, exist_ok=True)
        !gdown --id 1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684 -O {self.dataset_path}/data.zip
        !unzip -qq {self.dataset_path}/data.zip -d {self.dataset_path}

    def load_dataset(self):
        dataset = keras.utils.image_dataset_from_directory(
            self.dataset_path,
            label_mode=None,
            image_size=self.image_size,
            batch_size=self.batch_size,
            smart_resize=True
        )
        return dataset.map(lambda x: x / 255.0)

In [4]:
class Discriminator(keras.Model):
    def __init__(self, input_shape=(64, 64, 3)):
        super().__init__()
        self.model = keras.Sequential([
            layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Flatten(),
            layers.Dropout(0.2),
            layers.Dense(1, activation="sigmoid"),
        ], name="discriminator")

    def call(self, inputs):
        return self.model(inputs)

In [5]:
class Generator(keras.Model):
    def __init__(self, latent_dim=128):
        super().__init__()
        self.model = keras.Sequential([
            layers.Dense(8 * 8 * 128, input_shape=(latent_dim,)),
            layers.Reshape((8, 8, 128)),
            layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
        ], name="generator")

    def call(self, inputs):
        return self.model(inputs)

In [6]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        generated_images = self.generator(random_latent_vectors)
        combined_images = tf.concat([generated_images, real_images], axis=0)

        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        misleading_labels = tf.zeros((batch_size, 1))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}

In [7]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images = generated_images.numpy()

        for i in range(self.num_img):
            img = keras.utils.array_to_img(generated_images[i])
            img.save(f"generated_img_{epoch:03d}_{i}.png")

In [ ]:
if __name__ == "__main__":
    data_loader = CelebAGANDataLoader()
    dataset = data_loader.load_dataset()

    discriminator = Discriminator()
    generator = Generator(latent_dim=128)

    gan = GAN(discriminator=discriminator, generator=generator, latent_dim=128)
    gan.compile(
        d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss_fn=keras.losses.BinaryCrossentropy(),
    )

    gan.fit(dataset, epochs=100, callbacks=[GANMonitor(num_img=10, latent_dim=128)])